# City Planners guide to suggest amenity expansion based on growing population in Santa Clara County

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
# uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge requests --yes
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
# uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\harid\.conda\envs\TestEnv

  added / updated specs:
    - requests


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cffi-1.13.2                |   py37hb32ad35_0         220 KB  conda-forge
    chardet-3.0.4              |        py37_1003         184 KB  conda-forge
    cryptography-2.8           |   py37hb32ad35_1         564 KB  conda-forge
    idna-2.8                   |        py37_1000         100 KB  conda-forge
    pycparser-2.19             |           py37_1         171 KB  conda-forge
    pyopenssl-19.1.0           |           py37_0          83 KB  conda-forge
    pysocks-1.7.1              |           py37_0          27 KB  conda-forge
    requests-2.22.0            |           py37_1          84 KB  conda-forge
    urllib3-1.25.7             |           py37_0         159 KB  con

In [2]:
!curl -o uszips.zip "https://simplemaps.com/static/data/us-zips/1.7/basic/simplemaps_uszips_basicv1.7.zip"
!unzip uszips.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 3932k  100 3932k    0     0  3932k      0  0:00:01 --:--:--  0:00:01 5985k
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [481]:
!dir
#df_old = df_sc_copy #This is available as a backup
#df_old.head()

 Volume in drive C is Local Disk
 Volume Serial Number is 46E6-2CA0

 Directory of c:\Users\harid\IBM-Capstone-Project

01/28/2020  08:46 PM    <DIR>          .
01/28/2020  08:46 PM    <DIR>          ..
01/20/2020  05:28 PM    <DIR>          .ipynb_checkpoints
01/28/2020  08:46 PM        40,977,419 Coursera_capstone.ipynb
01/22/2020  01:29 PM           158,188 Extracting Table from Wiki - Project - Week 3.ipynb
01/20/2020  10:02 PM             2,891 Geospatial_Coordinates.csv
01/20/2020  02:25 PM             4,676 license.txt
01/25/2020  10:53 PM             1,325 README.md
01/20/2020  05:31 PM             9,029 scraped_table.tsv
01/20/2020  02:26 PM         5,998,595 uszips.csv
01/20/2020  02:26 PM         3,136,514 uszips.xlsx
01/26/2020  02:13 PM         4,027,097 uszips.zip
               9 File(s)     54,315,734 bytes
               3 Dir(s)  65,113,165,824 bytes free


In [497]:
df_zips=pd.read_csv('uszips.csv')
print(df_zips.columns)
print(df_zips.shape)


Index(['zip', 'lat', 'lng', 'city', 'state_id', 'state_name', 'zcta',
       'parent_zcta', 'population', 'density', 'county_fips', 'county_name',
       'county_weights', 'county_names_all', 'county_fips_all', 'imprecise',
       'military', 'timezone'],
      dtype='object')
(33099, 18)


In [498]:
df_sc = df_zips[df_zips.county_name == 'Santa Clara']

In [499]:
df_sc.head(10)
df_sc.shape

(57, 18)

In [500]:
df_sc = df_sc[['zip','lat','lng','city','population','density']]
print(df_sc.shape)
print(df_sc.head())

(57, 6)
         zip       lat        lng           city  population  density
30934  94022  37.35753 -122.14445      Los Altos       19378    408.9
30935  94024  37.35220 -122.09442      Los Altos       23961   1181.4
30942  94040  37.38058 -122.08535  Mountain View       35845   3454.9
30943  94041  37.38848 -122.07568  Mountain View       14394   3351.3
30944  94043  37.41751 -122.07039  Mountain View       31488   1025.5


In [501]:
CLIENT_ID = 'GNZIZNFO3YLF2ISTKOPOKHG2KJOLU1BE42QPDL0MTJZODMH5' # your Foursquare ID
CLIENT_SECRET = 'XF3OLFUNXY3IMRB4EUHOXBVZZCJOHAAV145I1NA0D5C3S4JH' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GNZIZNFO3YLF2ISTKOPOKHG2KJOLU1BE42QPDL0MTJZODMH5
CLIENT_SECRET:XF3OLFUNXY3IMRB4EUHOXBVZZCJOHAAV145I1NA0D5C3S4JH


In [502]:
# From https://developer.foursquare.com/docs/resources/categories
park_id = '4bf58dd8d48988d163941735'
library_id = '4bf58dd8d48988d12f941735'
fire_id ='4bf58dd8d48988d12c941735'
post_id = '4bf58dd8d48988d172941735'
# transit_ids = ['Bus station', 'Bus Stop', 'Train Station', 'Tram Station ']
transit_ids = '4bf58dd8d48988d1fe931735,52f2ab2ebcbc57f1066b8b4f,4bf58dd8d48988d129951735,52f2ab2ebcbc57f1066b8b51'
urgent_id = '56aa371be4b08b9a8d573526'
emergency_id = '4bf58dd8d48988d194941735'


In [503]:
col_labels = ['nearbyParks','nearbyLibraries','nearbyFireDept','nearbyPostOffice','nearbyTransit','nearbyUrgentCare','nearbyER']
df_sc = df_sc.reindex(df_sc.columns.tolist() + col_labels, axis=1)
df_sc.head()

,zip,lat,lng,city,population,density,nearbyParks,nearbyLibraries,nearbyFireDept,nearbyPostOffice,nearbyTransit,nearbyUrgentCare,nearbyER
30934,94022,37.35753,-122.14445,Los Altos,19378,408.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30935,94024,37.35220,-122.09442,Los Altos,23961,1181.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30942,94040,37.38058,-122.08535,Mountain View,35845,3454.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30943,94041,37.38848,-122.07568,Mountain View,14394,3351.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30944,94043,37.41751,-122.07039,Mountain View,31488,1025.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [504]:
#df_sc = df_sc.loc[:,~df_sc.columns.duplicated()]
#df_sc
df_sc_copy = df_sc.copy()
df_sc_copy.head()

,zip,lat,lng,city,population,density,nearbyParks,nearbyLibraries,nearbyFireDept,nearbyPostOffice,nearbyTransit,nearbyUrgentCare,nearbyER
30934,94022,37.35753,-122.14445,Los Altos,19378,408.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30935,94024,37.35220,-122.09442,Los Altos,23961,1181.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30942,94040,37.38058,-122.08535,Mountain View,35845,3454.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30943,94041,37.38848,-122.07568,Mountain View,14394,3351.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30944,94043,37.41751,-122.07039,Mountain View,31488,1025.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [505]:

#Block to return number of a particular public utility eg. Park, Library etc. within a specified radius.
def find_total_count(lat, lng, key_id, rad):
    #er = 0  #Use this only to computer ER/UC
    rad = rad*1609.34
    category_id = key_id
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&intent={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, key_id, rad, 'browse', LIMIT)
    result = requests.get(url).json()
    #print(result)
    for venue in result['response']['venues']:
        #if ' ER ' in venue['name'].upper() or ' EMERGENCY ' in venue['name'].upper():  #Use this only to computer ER/UC
        #if ' EXPRESS ' in venue['name'].upper() or ' URGENT ' in venue['name'].upper():  #Use this only to computer ER/UC
            print(venue['name'], venue['location']['distance'])
            #er+=1   #Use this only to computer ER/UC
    #val = er        #Use this only to computer ER/UC
    val = len(result['response']['venues'])    
    return(val)



In [455]:
# From the sources of information , assigning the value of :
#'nearby parks'= 0.5 miles
#'nearby libraries = 2.1 miles'
#'nearby Fire Station = 10 miles'
#'nearby Postal dept & Transit stations = 5 miles'
#'nearby Urgent care = 6.6 miles'
#'nearby Emergency Room = 3.7 miles'
for index, row in df_sc_copy.iterrows():
    df_sc_copy.loc[index, 'nearbyParks'] = find_total_count(row['lat'],row['lng'],park_id,1)
    df_sc_copy.loc[index, 'nearbyFireDept'] = find_total_count(row['lat'],row['lng'],fire_id,10.0)
    df_sc_copy.loc[index, 'nearbyLibraries'] = find_total_count(row['lat'],row['lng'],library_id,2.1)
    df_sc_copy.loc[index, 'nearbyPostOffice'] = find_total_count(row['lat'],row['lng'],post_id,5)
    df_sc_copy.loc[index, 'nearbyTransit'] = find_total_count(row['lat'],row['lng'],transit_ids,5)
    df_sc_copy.loc[index, 'nearbyUrgentCare'] = find_total_count(row['lat'],row['lng'],urgent_id,6.6)
    df_sc_copy.loc[index, 'nearbyER'] = find_total_count(row['lat'],row['lng'],emergency_id ,3.7)
df_sc_copy.head()


Los Altos Hills Byrne Preserve 877
McKenzie Park 765
Marymeade Park 1329
Heritage Oaks Park 1032
Montclaire Park 1539
The Swing 914
Eagle Park 942
Pioneer Park 1002
Cuesta Park 971
Mercy-Bush Park 1109
The Benches @ Willow Pavilion 1273
Dana Park 1515
Bubb Park 374
The Most Exciting Park In The World Of Ever 1207
Varsity Park 732
Gateway Park 609
Landels Park 1505
Fairmont Park 640
Eagle Park 884
Pioneer Park 697
Slater School Park 1347
Sylvan Park 1371
Creekside Park 1141
Mercy-Bush Park 188
Jackson Park 1106
Dana Park 911
Bubb Park 1328
Mariposa Avenue Park 1409
The Most Exciting Park In The World Of Ever 196
Shenandoah Square Apartments 1483
Devonshire Park 1469
Gateway Park 749
Whisman Park 1514
Landels Park 423
Moffett Park 1535
Fairmont Park 587
Charleston Park 1162
Shoreline Kite Flying Area 1514
Shoreline Business Park 717
San Veron Park 1518
Stierlin Ct Park 667
Boat Dodo 1558
A.R.C. Park 1198
Fair Oaks Park 392
Seven Seas Neighborhood Park 1123
Swegles Park 1253
John W. Chris

,zip,lat,lng,city,population,density,nearbyParks,nearbyFireDept,nearbyLibraries,nearbyER,nearbyUrgentCare,nearbyPostOffice,nearbyTransit,cluster_nearbyParks,cluster_nearbyLibraries,cluster_nearbyFireDept,cluster_nearbyPostOffice,cluster_nearbyTransit,cluster_nearbyUrgentCare,cluster_nearbyER,cluster_all
30934,94022,37.35753,-122.14445,Los Altos,19378,408.9,1.0,48.0,0.0,1.0,7.0,18.0,50.0,0,0,0,2,0,2,2,2
30935,94024,37.35220,-122.09442,Los Altos,23961,1181.4,5.0,50.0,6.0,0.0,6.0,30.0,50.0,0,2,0,0,0,2,1,2
30942,94040,37.38058,-122.08535,Mountain View,35845,3454.9,12.0,50.0,10.0,1.0,9.0,38.0,50.0,2,2,0,1,0,2,2,2
30943,94041,37.38848,-122.07568,Mountain View,14394,3351.3,18.0,50.0,10.0,1.0,9.0,37.0,50.0,2,2,0,0,0,2,2,2
30944,94043,37.41751,-122.07039,Mountain View,31488,1025.5,7.0,50.0,5.0,0.0,9.0,34.0,50.0,0,2,0,0,0,2,1,2


In [511]:
#df_sc_copy.drop(df_sc_copy.columns[[range(6,32)]], axis=1, inplace=True)
df_sc.head()

,zip,lat,lng,city,population,density,nearbyParks,nearbyFireDept,nearbyLibraries,nearbyER,nearbyUrgentCare,nearbyPostOffice,nearbyTransit,cluster_nearbyParks,cluster_nearbyLibraries,cluster_nearbyFireDept,cluster_nearbyPostOffice,cluster_nearbyTransit,cluster_nearbyUrgentCare,cluster_nearbyER,cluster_all
30934,94022,37.35753,-122.14445,Los Altos,19378,408.9,1.0,48.0,0.0,1.0,7.0,18.0,50.0,0,1,0,0,0,2,2,1
30935,94024,37.35220,-122.09442,Los Altos,23961,1181.4,5.0,50.0,6.0,0.0,6.0,30.0,50.0,1,3,0,1,0,2,0,3
30942,94040,37.38058,-122.08535,Mountain View,35845,3454.9,12.0,50.0,10.0,1.0,9.0,38.0,50.0,2,0,0,1,0,2,2,3
30943,94041,37.38848,-122.07568,Mountain View,14394,3351.3,18.0,50.0,10.0,1.0,9.0,37.0,50.0,3,0,0,1,0,2,2,3
30944,94043,37.41751,-122.07039,Mountain View,31488,1025.5,7.0,50.0,5.0,0.0,9.0,34.0,50.0,1,3,0,1,0,2,0,3


In [520]:
# run k-means clustering
kclusters = 4
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pd.DataFrame(df_sc['nearbyParks']))
print("Parks Cluster:",kmeans.labels_)
df_sc['cluster_nearbyParks'] = kmeans.labels_
      
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pd.DataFrame(df_sc['nearbyLibraries']))
print("Libraries Cluster:",kmeans.labels_)
df_sc['cluster_nearbyLibraries'] = kmeans.labels_
      
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pd.DataFrame(df_sc['nearbyFireDept']))
print("Fire Station Cluster:",kmeans.labels_)
df_sc['cluster_nearbyFireDept'] = kmeans.labels_
      
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pd.DataFrame(df_sc['nearbyPostOffice']))
print("Postal Service Cluster:",kmeans.labels_)
df_sc['cluster_nearbyPostOffice'] = kmeans.labels_
      
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pd.DataFrame(df_sc['nearbyTransit']))
print("Transport_Score Cluster:",kmeans.labels_)
df_sc['cluster_nearbyTransit'] = kmeans.labels_
      
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pd.DataFrame(df_sc['nearbyUrgentCare']))
print("UrgentCare Cluster:",kmeans.labels_)
df_sc['cluster_nearbyUrgentCare'] = kmeans.labels_
      
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pd.DataFrame(df_sc['nearbyER']))
print("EmergencyRoom Cluster:",kmeans.labels_)
df_sc['cluster_nearbyER'] = kmeans.labels_

# run k-means clustering on all the data points above
kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pd.DataFrame(df_sc[['nearbyParks', 'nearbyLibraries','nearbyFireDept','nearbyPostOffice','nearbyTransit','nearbyUrgentCare','nearbyER']]))
print("Cluster of all params:",kmeans.labels_)
df_sc['cluster_all'] = kmeans.labels_


Parks Cluster: [0 1 2 3 1 1 1 2 0 2 0 1 2 0 2 0 0 0 1 0 0 0 0 2 1 1 2 0 1 0 3 3 0 1 0 1 0
 1 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0]
Libraries Cluster: [1 3 0 0 3 3 3 3 1 0 1 0 0 1 3 1 1 3 3 1 3 1 1 0 3 0 1 1 2 3 2 2 0 0 3 3 1
 3 3 3 3 3 2 1 0 3 1 3 1 0 3 1 3 1 1 1 1]
Fire Station Cluster: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 2 3 3 0 2 2 0 0 0 0 3 0 0 0 0 0 0 0 1 1
 0 0 3 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 2 3]
Postal Service Cluster: [0 1 1 1 1 2 2 2 2 1 0 1 1 0 1 3 0 3 0 0 0 3 3 2 2 2 2 0 2 1 1 2 1 1 1 3 3
 1 1 0 1 2 2 0 2 1 1 1 3 1 1 3 1 3 3 3 0]
Transport_Score Cluster: [0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 3 0 2 0 1 1 1 1 1 0 0 1 0 0 0 0 0 2 2 1
 0 0 0 2 0 1 0 0 0 0 0 3 0 0 3 0 1 1 3 0]
UrgentCare Cluster: [2 2 2 2 2 1 1 1 1 2 2 2 2 1 1 0 2 0 2 2 2 0 0 3 3 3 3 2 3 0 1 1 1 3 2 0 0
 0 2 0 2 1 3 0 3 3 1 1 0 1 1 0 2 0 0 0 0]
EmergencyRoom Cluster: [2 0 2 2 0 2 2 2 0 1 1 1 1 0 3 2 0 0 2 2 0 0 0 3 3 1 0 0 3 1 3 3 2 3 1 1 0
 2 1 1 2 3 3 2 1 3 3 2 0 2 0 0 1 0 2 0 0]
Cluster of all params: [3 0 4 4 0

In [458]:
print(df_sc[['nearbyParks','cluster_nearbyParks']].sort_values(by='nearbyParks'))

       nearbyParks  cluster_nearbyParks
31211          0.0                    0
31165          0.0                    0
31209          0.0                    0
31207          0.0                    0
31206          0.0                    0
31204          0.0                    0
31162          0.0                    0
31199          0.0                    0
31170          0.0                    0
31145          0.0                    0
31155          1.0                    0
31154          1.0                    0
31192          1.0                    0
30934          1.0                    0
30989          1.0                    0
31197          2.0                    0
31159          2.0                    0
31210          2.0                    0
31195          2.0                    0
31202          3.0                    0
31190          3.0                    0
31203          3.0                    0
31188          3.0                    0
31194          3.0                    0


In [459]:
# Data Viz
import matplotlib.cm as cm
import matplotlib.colors as colors
address='San Jose, CA'

geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Cluster order (least to most ERs): 1,3,2,0
radii = {
    0: 2,
    1: 5,
    2: 8,
    3: 12
}

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_sc['lat'], df_sc['lng'], df_sc['zip'], df_sc['cluster_nearbyParks']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=radii[cluster],
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [460]:
print(df_sc[['nearbyLibraries','cluster_nearbyLibraries']].sort_values(by='nearbyLibraries'))

       nearbyLibraries  cluster_nearbyLibraries
30934              0.0                        1
31155              0.0                        1
31154              0.0                        1
31145              0.0                        1
31180              0.0                        1
30989              0.0                        1
31165              0.0                        1
31211              0.0                        1
31204              0.0                        1
31207              0.0                        1
31209              0.0                        1
31192              0.0                        1
31162              0.0                        1
30958              1.0                        1
31170              1.0                        1
31199              2.0                        1
31202              2.0                        1
31174              2.0                        1
31210              2.0                        1
31212              2.0                  

In [461]:
# Data Viz
import matplotlib.cm as cm
import matplotlib.colors as colors
address='San Jose, CA'

geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Cluster order (least to most ERs): 1,3,2,0
radii = {
    1: 2,
    3: 5,
    0: 8,
    2: 12
}

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_sc['lat'], df_sc['lng'], df_sc['zip'], df_sc['cluster_nearbyLibraries']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=radii[cluster],
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [462]:
print(df_sc[['nearbyFireDept','cluster_nearbyFireDept']].sort_values(by='nearbyFireDept'))

       nearbyFireDept  cluster_nearbyFireDept
31211             2.0                       2
31165             4.0                       2
31159             5.0                       2
31170             5.0                       2
31207            18.0                       1
31210            19.0                       1
31209            19.0                       1
31154            19.0                       1
31204            24.0                       1
31192            26.0                       1
31191            28.0                       1
31212            41.0                       3
31161            42.0                       3
31195            42.0                       3
31180            42.0                       3
31162            44.0                       3
30989            47.0                       0
31185            48.0                       0
30934            48.0                       0
30990            49.0                       0
31151            50.0             

In [463]:
# Data Viz
import matplotlib.cm as cm
import matplotlib.colors as colors
address='San Jose, CA'

geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Cluster order (least to most ERs): 1,3,2,0
radii = {
    2: 2,
    1: 5,
    3: 8,
    0: 12
}

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_sc['lat'], df_sc['lng'], df_sc['zip'], df_sc['cluster_nearbyFireDept']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=radii[cluster],
        popup=label,
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [464]:
print(df_sc[['nearbyPostOffice','cluster_nearbyPostOffice']].sort_values(by='nearbyPostOffice'))

       nearbyPostOffice  cluster_nearbyPostOffice
31207               0.0                         3
31204               0.0                         3
31211               0.0                         3
31209               1.0                         3
31154               2.0                         3
31159               3.0                         3
31165               3.0                         3
31210               4.0                         3
31192               5.0                         3
31170               5.0                         3
31191               7.0                         3
31180              11.0                         0
31212              12.0                         0
31161              14.0                         0
31164              15.0                         0
31162              15.0                         0
31199              15.0                         0
31155              15.0                         0
30989              17.0                         0


In [465]:
# Data Viz
import matplotlib.cm as cm
import matplotlib.colors as colors
address='San Jose, CA'

geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Cluster order (least to most ERs): 1,3,2,0
radii = {
    3: 2,
    0: 5,
    1: 8,
    2: 12
}

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_sc['lat'], df_sc['lng'], df_sc['zip'], df_sc['cluster_nearbyPostOffice']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=radii[cluster],
        popup=label,
        color='dark blue',
        fill=True,
        fill_color='dark blue',
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [466]:
print(df_sc[['nearbyTransit','cluster_nearbyTransit']].sort_values(by='nearbyTransit'))

       nearbyTransit  cluster_nearbyTransit
31211            0.0                      3
31204            0.0                      3
31207            3.0                      3
31159            8.0                      3
31154           11.0                      1
31165           13.0                      1
31209           14.0                      1
31198           15.0                      1
31192           16.0                      1
30955           16.0                      1
30956           16.0                      1
31172           17.0                      1
31184           17.0                      1
31170           18.0                      1
31210           20.0                      1
31173           20.0                      1
31171           22.0                      1
31191           33.0                      2
31190           38.0                      2
31162           40.0                      2
31196           43.0                      2
31180           48.0            

In [467]:
# Experiment
# Data Viz
import matplotlib.cm as cm
import matplotlib.colors as colors
address='San Jose, CA'

geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Cluster order (least to most ERs): 1,3,2,0
radii = {
    3: 2,
    1: 5,
    2: 8,
    0: 12
}

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_sc['lat'], df_sc['lng'], df_sc['zip'], df_sc['cluster_nearbyTransit']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=radii[cluster],
        popup=label,
        color='yellow',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [469]:
print(df_sc[['nearbyUrgentCare','cluster_nearbyUrgentCare']].sort_values(by='nearbyUrgentCare'))

       nearbyUrgentCare  cluster_nearbyUrgentCare
31212               0.0                         0
31204               0.0                         0
31211               0.0                         0
31159               0.0                         0
31210               1.0                         0
31209               1.0                         0
31207               1.0                         0
31191               1.0                         0
31192               1.0                         0
31170               2.0                         0
31154               2.0                         0
31193               2.0                         0
31165               2.0                         0
31195               3.0                         0
31199               3.0                         0
31185               4.0                         0
31194               5.0                         2
30935               6.0                         2
30989               6.0                         2


In [470]:
# Experiment
# Data Viz
import matplotlib.cm as cm
import matplotlib.colors as colors
address='San Jose, CA'

geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Cluster order (least to most ERs): 1,3,2,0
radii = {
    0: 2,
    2: 5,
    1: 8,
    3: 12
}

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_sc['lat'], df_sc['lng'], df_sc['zip'], df_sc['cluster_nearbyUrgentCare']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=radii[cluster],
        popup=label,
        color='#ff0000',
        fill=True,
        fill_color='#ff0000',
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [472]:
print(df_sc[['nearbyER','cluster_nearbyER']].sort_values(by='nearbyER'))

       nearbyER  cluster_nearbyER
31212       0.0                 0
31170       0.0                 0
31165       0.0                 0
31164       0.0                 0
31211       0.0                 0
31159       0.0                 0
31155       0.0                 0
31192       0.0                 0
31145       0.0                 0
31180       0.0                 0
31174       0.0                 0
31204       0.0                 0
30958       0.0                 0
31206       0.0                 0
31207       0.0                 0
30944       0.0                 0
31209       0.0                 0
30935       0.0                 0
31199       1.0                 2
31193       1.0                 2
31205       1.0                 2
31188       1.0                 2
31210       1.0                 2
31203       1.0                 2
31196       1.0                 2
30934       1.0                 2
31154       1.0                 2
30942       1.0                 2
31162       1.

In [473]:
# Experiment
# Data Viz
import matplotlib.cm as cm
import matplotlib.colors as colors
address='San Jose, CA'

geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Cluster order (least to most ERs): 1,3,2,0
radii = {
    0: 2,
    2: 5,
    1: 8,
    3: 12
}

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_sc['lat'], df_sc['lng'], df_sc['zip'], df_sc['cluster_nearbyER']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=radii[cluster],
        popup=label,
        color='#ff0000',
        fill=True,
        fill_color='#ff0000',
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [474]:
df_sc

,zip,lat,lng,city,population,density,nearbyParks,nearbyFireDept,nearbyLibraries,nearbyER,nearbyUrgentCare,nearbyPostOffice,nearbyTransit,cluster_nearbyParks,cluster_nearbyLibraries,cluster_nearbyFireDept,cluster_nearbyPostOffice,cluster_nearbyTransit,cluster_nearbyUrgentCare,cluster_nearbyER,cluster_all
30934,94022,37.35753,-122.14445,Los Altos,19378,408.9,1.0,48.0,0.0,1.0,7.0,18.0,50.0,0,1,0,0,0,2,2,1
30935,94024,37.35220,-122.09442,Los Altos,23961,1181.4,5.0,50.0,6.0,0.0,6.0,30.0,50.0,1,3,0,1,0,2,0,3
30942,94040,37.38058,-122.08535,Mountain View,35845,3454.9,12.0,50.0,10.0,1.0,9.0,38.0,50.0,2,0,0,1,0,2,2,3
30943,94041,37.38848,-122.07568,Mountain View,14394,3351.3,18.0,50.0,10.0,1.0,9.0,37.0,50.0,3,0,0,1,0,2,2,3
30944,94043,37.41751,-122.07039,Mountain View,31488,1025.5,7.0,50.0,5.0,0.0,9.0,34.0,50.0,1,3,0,1,0,2,0,3
30955,94085,37.38854,-122.01777,Sunnyvale,23612,2349.8,10.0,50.0,5.0,1.0,14.0,46.0,16.0,1,3,0,2,1,1,2,2
30956,94086,37.37167,-122.02312,Sunnyvale,49630,3930.6,8.0,50.0,5.0,1.0,14.0,49.0,16.0,1,3,0,2,1,1,2,2
30957,94087,37.35153,-122.03692,Sunnyvale,57219,3184.4,11.0,50.0,4.0,1.0,13.0,47.0,50.0,2,3,0,2,0,1,2,3
30958,94089,37.41154,-122.01404,Sunnyvale,22313,1240.2,4.0,50.0,1.0,0.0,14.0,43.0,50.0,0,1,0,2,0,1,0,3
30987,94301,37.44412,-122.14995,Palo Alto,17191,2767.2,11.0,50.0,12.0,2.0,8.0,25.0,50.0,2,0,0,1,0,2,1,1


In [521]:
# Data Viz
import matplotlib.cm as cm
import matplotlib.colors as colors
address='San Jose, CA'

geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# Cluster order (least to most ERs): 1,3,2,0
radii = {
    1: 2,
    3: 5,
    2: 8,
    4: 12,
    0: 15
}
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_sc['lat'], df_sc['lng'], df_sc['zip'], df_sc['cluster_all']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=radii[cluster],
        popup=label,
        color='violet',
        fill=True,
        fill_color='violet',
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [522]:
df_sc[['city','nearbyParks', 'nearbyLibraries','nearbyFireDept','nearbyPostOffice','nearbyTransit','nearbyUrgentCare','nearbyER','cluster_all']]

,city,nearbyParks,nearbyLibraries,nearbyFireDept,nearbyPostOffice,nearbyTransit,nearbyUrgentCare,nearbyER,cluster_all
30934,Los Altos,1.0,0.0,48.0,18.0,50.0,7.0,1.0,3
30935,Los Altos,5.0,6.0,50.0,30.0,50.0,6.0,0.0,0
30942,Mountain View,12.0,10.0,50.0,38.0,50.0,9.0,1.0,4
30943,Mountain View,18.0,10.0,50.0,37.0,50.0,9.0,1.0,4
30944,Mountain View,7.0,5.0,50.0,34.0,50.0,9.0,0.0,0
30955,Sunnyvale,10.0,5.0,50.0,46.0,16.0,14.0,1.0,2
30956,Sunnyvale,8.0,5.0,50.0,49.0,16.0,14.0,1.0,2
30957,Sunnyvale,11.0,4.0,50.0,47.0,50.0,13.0,1.0,4
30958,Sunnyvale,4.0,1.0,50.0,43.0,50.0,14.0,0.0,4
30987,Palo Alto,11.0,12.0,50.0,25.0,50.0,8.0,2.0,0


Sources:
    http://cloud.tpl.org/pubs/ccpe_Planning_mag_article12_2004.pdf
    https://atcoordinates.info/2016/02/22/average-distance-to-public-libraries-in-the-us/
    https://www.planning.org/pas/reports/report98.htm
    https://www.health.org.uk/sites/default/files/QualityWatch_FocusOnDistanceFromHomeToEmergencyCare.pdf
    https://www.ofm.wa.gov/sites/default/files/public/legacy/researchbriefs/2013/brief070.pdf
    